# import datasets

In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from catboost import CatBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sb

train_X = pd.read_csv('./X_train_imputed.csv')
train_y = pd.read_csv('./y_train.csv')
test_X = pd.read_csv('./X_test_imputed.csv')

# split the data for initial training on the trained_data

In [2]:
train_X = train_X.drop(['respondent_id'], axis=1)
X = train_X
y = train_y[['h1n1_vaccine','seasonal_vaccine']]

In [3]:
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Make ANN model 

In [4]:
from sklearn.linear_model import Perceptron
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(2, activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

In [5]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/100
268/268 [==============================] - 2s 5ms/step - loss: 0.6517 - accuracy: 0.5267 - val_loss: 0.4761 - val_accuracy: 0.4959 - lr: 3.0000e-04
Epoch 2/100
268/268 [==============================] - 1s 5ms/step - loss: 0.5207 - accuracy: 0.5190 - val_loss: 0.4395 - val_accuracy: 0.5008 - lr: 3.0000e-04
Epoch 3/100
268/268 [==============================] - 1s 5ms/step - loss: 0.4829 - accuracy: 0.4810 - val_loss: 0.4233 - val_accuracy: 0.4613 - lr: 3.0000e-04
Epoch 4/100
268/268 [==============================] - 1s 5ms/step - loss: 0.4608 - accuracy: 0.4520 - val_loss: 0.4171 - val_accuracy: 0.4056 - lr: 3.0000e-04
Epoch 5/100
268/268 [==============================] - 1s 4ms/step - loss: 0.4473 - accuracy: 0.4323 - val_loss: 0.4113 - val_accuracy: 0.3742 - lr: 3.0000e-04
Epoch 6/100
268/268 [==============================] - 1s 4ms/step - loss: 0.4399 - accuracy: 0.4083 - val_loss: 0.4093 - val_accuracy: 0.3665 - lr: 3.0000e-04
Epoch 7/100
268/268 [===================

Epoch 52/100
268/268 [==============================] - 1s 4ms/step - loss: 0.3894 - accuracy: 0.3953 - val_loss: 0.4097 - val_accuracy: 0.3513 - lr: 1.0000e-05
Epoch 53/100
268/268 [==============================] - 1s 4ms/step - loss: 0.3877 - accuracy: 0.3963 - val_loss: 0.4101 - val_accuracy: 0.3510 - lr: 1.0000e-05
Epoch 54/100
268/268 [==============================] - 1s 5ms/step - loss: 0.3887 - accuracy: 0.3965 - val_loss: 0.4099 - val_accuracy: 0.3531 - lr: 1.0000e-05
Epoch 55/100
268/268 [==============================] - 1s 4ms/step - loss: 0.3852 - accuracy: 0.3974 - val_loss: 0.4099 - val_accuracy: 0.3522 - lr: 1.0000e-05
Epoch 56/100
268/268 [==============================] - 1s 4ms/step - loss: 0.3882 - accuracy: 0.3947 - val_loss: 0.4101 - val_accuracy: 0.3557 - lr: 1.0000e-05
Epoch 57/100
268/268 [==============================] - 1s 4ms/step - loss: 0.3893 - accuracy: 0.3954 - val_loss: 0.4099 - val_accuracy: 0.3524 - lr: 1.0000e-05
Epoch 58/100
268/268 [============

In [6]:
# Save the trained model
model.save('ann_model.h5')

In [7]:
# Evaluating the model
loss = model.evaluate(X_test, y_test)
print(f'Model Loss: {loss}')

# Making predictions
yhat = model.predict(X_test)

167/167 [==============================] - 0s 1ms/step - loss: 0.4147 - accuracy: 0.3562
Model Loss: [0.41471579670906067, 0.3562336266040802]


# Now train entire train_data and predict the test_targets

In [8]:
X_pred = test_X

New_df = pd.DataFrame(X_pred['respondent_id'])
New_df

respondent_id
0              26707
1              26708
2              26709
3              26710
4              26711
...              ...
26703          53410
26704          53411
26705          53412
26706          53413
26707          53414

[26708 rows x 1 columns]

In [9]:
X_test3 = X_pred.drop(['respondent_id'], axis = 1)

X_train_set = X
y_train_set = y

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
X_train_set = scaler.fit_transform(X_train_set)
X_test3 = scaler.fit_transform(X_test3)

In [11]:
history = model.fit(X_train, y_train, epochs=200, batch_size=96, validation_split=0.2, callbacks=[reduce_lr])

Epoch 1/200
179/179 [==============================] - 2s 7ms/step - loss: 0.4479 - accuracy: 0.4110 - val_loss: 0.5697 - val_accuracy: 0.3548 - lr: 1.0000e-05
Epoch 2/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4444 - accuracy: 0.4041 - val_loss: 0.4261 - val_accuracy: 0.3719 - lr: 1.0000e-05
Epoch 3/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4409 - accuracy: 0.4016 - val_loss: 0.4149 - val_accuracy: 0.3777 - lr: 1.0000e-05
Epoch 4/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4368 - accuracy: 0.4015 - val_loss: 0.4121 - val_accuracy: 0.3749 - lr: 1.0000e-05
Epoch 5/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4377 - accuracy: 0.4014 - val_loss: 0.4105 - val_accuracy: 0.3751 - lr: 1.0000e-05
Epoch 6/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4355 - accuracy: 0.3938 - val_loss: 0.4086 - val_accuracy: 0.3737 - lr: 1.0000e-05
Epoch 7/200
179/179 [===================

Epoch 52/200
179/179 [==============================] - 1s 5ms/step - loss: 0.4113 - accuracy: 0.3735 - val_loss: 0.3982 - val_accuracy: 0.3436 - lr: 1.0000e-05
Epoch 53/200
179/179 [==============================] - 1s 5ms/step - loss: 0.4108 - accuracy: 0.3698 - val_loss: 0.3983 - val_accuracy: 0.3457 - lr: 1.0000e-05
Epoch 54/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4135 - accuracy: 0.3711 - val_loss: 0.3983 - val_accuracy: 0.3436 - lr: 1.0000e-05
Epoch 55/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4099 - accuracy: 0.3724 - val_loss: 0.3984 - val_accuracy: 0.3461 - lr: 1.0000e-05
Epoch 56/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4126 - accuracy: 0.3706 - val_loss: 0.3981 - val_accuracy: 0.3454 - lr: 1.0000e-05
Epoch 57/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4101 - accuracy: 0.3694 - val_loss: 0.3977 - val_accuracy: 0.3431 - lr: 1.0000e-05
Epoch 58/200
179/179 [============

Epoch 103/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4019 - accuracy: 0.3762 - val_loss: 0.3979 - val_accuracy: 0.3492 - lr: 1.0000e-05
Epoch 104/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4015 - accuracy: 0.3768 - val_loss: 0.3981 - val_accuracy: 0.3508 - lr: 1.0000e-05
Epoch 105/200
179/179 [==============================] - 1s 5ms/step - loss: 0.4021 - accuracy: 0.3744 - val_loss: 0.3975 - val_accuracy: 0.3492 - lr: 1.0000e-05
Epoch 106/200
179/179 [==============================] - 1s 6ms/step - loss: 0.3992 - accuracy: 0.3763 - val_loss: 0.3974 - val_accuracy: 0.3485 - lr: 1.0000e-05
Epoch 107/200
179/179 [==============================] - 1s 6ms/step - loss: 0.3999 - accuracy: 0.3794 - val_loss: 0.3978 - val_accuracy: 0.3485 - lr: 1.0000e-05
Epoch 108/200
179/179 [==============================] - 1s 6ms/step - loss: 0.4037 - accuracy: 0.3736 - val_loss: 0.3976 - val_accuracy: 0.3485 - lr: 1.0000e-05
Epoch 109/200
179/179 [=====

179/179 [==============================] - 1s 6ms/step - loss: 0.3944 - accuracy: 0.3786 - val_loss: 0.3994 - val_accuracy: 0.3517 - lr: 1.0000e-05
Epoch 154/200
179/179 [==============================] - 1s 6ms/step - loss: 0.3962 - accuracy: 0.3788 - val_loss: 0.3995 - val_accuracy: 0.3494 - lr: 1.0000e-05
Epoch 155/200
179/179 [==============================] - 1s 5ms/step - loss: 0.3939 - accuracy: 0.3747 - val_loss: 0.3995 - val_accuracy: 0.3492 - lr: 1.0000e-05
Epoch 156/200
179/179 [==============================] - 1s 6ms/step - loss: 0.3962 - accuracy: 0.3771 - val_loss: 0.3992 - val_accuracy: 0.3494 - lr: 1.0000e-05
Epoch 157/200
179/179 [==============================] - 1s 5ms/step - loss: 0.3933 - accuracy: 0.3778 - val_loss: 0.3995 - val_accuracy: 0.3501 - lr: 1.0000e-05
Epoch 158/200
179/179 [==============================] - 1s 6ms/step - loss: 0.3929 - accuracy: 0.3752 - val_loss: 0.3996 - val_accuracy: 0.3517 - lr: 1.0000e-05
Epoch 159/200
179/179 [===================

# Make a new dataset ready for submission

In [12]:
X_test3 = X_pred.drop(['respondent_id'], axis = 1)
yhatt = model.predict(X_test3)

New_df[['h1n1_vaccine','seasonal_vaccine']] = yhatt
New_df.to_csv('./results_catboost.csv', index=False)